In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
from nltk.corpus import stopwords
from scipy.stats import skew, kurtosis
import re
from gensim.models import Word2Vec
import gensim
from gensim.models.wrappers import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
from gensim.models import KeyedVectors

Word2Vec = KeyedVectors.load_word2vec_format('../diff_traditional/source_data/wiki.es.vec', binary=False)

/home/nikuailemateam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
train = pd.read_csv('preprocessing_train.csv')
test = pd.read_csv('preprocessing_test.csv')
test['label'] = -1
test['test_id'] = range(len(test))

In [29]:
train.head(20)

,label,spa_qura1,spa_qura2
0,0,hola hago clic en el producto recibido,Compré un producto y no he recibido un correo ...
1,0,Hola Cerré la disputa el 21 de mayo de 2017 y ...,No obtuve el reembolso de mi dinero Han pasado...
2,0,Ordené de España a España ahora que mandan el ...,Mi pedido llegó pero el color es diferente al ...
3,1,Debo pagar impuestos personalizados,Cómo pagar los derechos de aduana
4,0,No recibí mi pedido,Mi pedido muestra que no he pagado pero lo hice
5,0,Hola Eva Al principio la pregunta Puedo escrib...,Cómo puedo escribir comentarios para los produ...
6,0,estoy tan enojado,Abrí una disputa y hasta ahora nada
7,0,No puedo abrir una disputa,No puedo rastrear
8,0,Cómo puedo verificar el número de pedido,Dónde puedo ver el número de mi rastreador de ...
9,0,No necesito una respuesta tuya,Necesito una factura por este artículo que te ...


In [9]:
print ("去掉未在word2vec中的词...")
def spa_norm_word_list(s1):
    s1 = str(s1).lower().split()
    s1 = [w for w in s1 if w in Word2Vec]
    return s1

train['spa_qura1_list'] = train['spa_qura1'].apply(spa_norm_word_list)
train['spa_qura2_list'] = train['spa_qura2'].apply(spa_norm_word_list)

test['spa_qura1_list'] = test['spa_qura1'].apply(spa_norm_word_list)
test['spa_qura2_list'] = test['spa_qura2'].apply(spa_norm_word_list)

print("去掉停用词结束...")

去掉未在word2vec中的词...
去掉停用词结束...


In [10]:
print("生成embedding...")

spa_list = list(train['spa_qura1'])
spa_list.extend(list(train['spa_qura2']))
spa_list.extend(list(test['spa_qura1']))
spa_list.extend(list(test['spa_qura2']))

MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 55
EMBEDDING_DIM = 300

# 生成token词典
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(spa_list)

texts_1 = list(train['spa_qura1'])
texts_2 = list(train['spa_qura2'])
test_texts_1 = list(test['spa_qura1'])
test_texts_2 = list(test['spa_qura2'])

# 得到词索引,相当于把每个sentence给序列化
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

# number of unique words
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

# 长度不足max_length的补0
data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index)) + 1
print('There are %d words' % nb_words)

# 把每个单词变成维度为EMBEDDING_DIM的向量,向量根据word2vec得到
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in Word2Vec:
        embedding_matrix[i] = Word2Vec[word]
# 查看哪些词没有词向量
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

生成embedding...
Found 5991 unique tokens
Preparing embedding matrix
There are 5992 words
Null word embeddings: 693


In [5]:
#获取distance_feature
df = pd.read_csv('../diff_traditional/train.dat')
distance_feature = df[['len_char_s1','len_char_s2','fuzz_QRatio','fuzz_WRatio']]
distance_feature = np.array(distance_feature)
test_distance_feature = distance_feature[21400:]

In [11]:
from cnn_model_3 import *

In [12]:
#from cnn_model import *
#from matchpyramid_model import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
print ("Running model")

# label = list(train_e.label)
# label.extend(list(train_s.label))
label = np.array(list(train.label))
# label = np.array(label)

folds = KFold(n_splits = 5, shuffle = False, random_state = 123)
pred_results = []
re_weight = True
val_loss_result = []
train_pred_results = []
for idx_train, idx_val in folds.split(data_1):
    data_1_train = data_1[idx_train]
    data_2_train = data_2[idx_train]
    #dis_train = distance_feature[idx_train]
    labels_train = label[idx_train]

    data_1_val = data_1[idx_val]
    data_2_val = data_2[idx_val]
    #dis_val = distance_feature[idx_val]
    labels_val = label[idx_val]
    
    Cnn = CNN3()
    model = Cnn.build_model(embedding_matrix, data_1.shape[1])
#     matchPyramid = MatchPyramid()
#     model = matchPyramid.build_model(embedding_matrix, data_1.shape[1])

    early_stopping = EarlyStopping(monitor='val_loss', patience=1)
    
    hist = model.fit([data_1_train, data_2_train],
                     labels_train, 
                     validation_data=([data_1_val, data_2_val], labels_val),
                     epochs=25,
                     batch_size=128,
                     shuffle=True, 
                     callbacks=[early_stopping],
                     verbose = 2)
    
    bst_val_score = min(hist.history['val_loss'])
    val_loss_result.append(bst_val_score)
    #model.load_weights("weights-improvement.hdf5")
    preds = model.predict([test_data_1, test_data_2], batch_size = 2048, verbose = 2)
    train_preds = model.predict([data_1_val, data_2_val], batch_size = 2048, verbose = 2)

    pred_results.append(preds)
    train_pred_results.extend(train_preds)

Running model
Train on 17120 samples, validate on 4280 samples
Epoch 1/25
 - 171s - loss: 0.5889 - acc: 0.7134 - val_loss: 0.4745 - val_acc: 0.7897
Epoch 2/25
 - 163s - loss: 0.4517 - acc: 0.8049 - val_loss: 0.4188 - val_acc: 0.8215
Epoch 3/25
 - 163s - loss: 0.3734 - acc: 0.8379 - val_loss: 0.4031 - val_acc: 0.8304
Epoch 4/25
 - 163s - loss: 0.3193 - acc: 0.8621 - val_loss: 0.3764 - val_acc: 0.8423
Epoch 5/25
 - 163s - loss: 0.2725 - acc: 0.8862 - val_loss: 0.4176 - val_acc: 0.8493
Train on 17120 samples, validate on 4280 samples
Epoch 1/25
 - 169s - loss: 0.5873 - acc: 0.7178 - val_loss: 0.4929 - val_acc: 0.7958
Epoch 2/25
 - 161s - loss: 0.4520 - acc: 0.8041 - val_loss: 0.4202 - val_acc: 0.8206
Epoch 3/25
 - 161s - loss: 0.3834 - acc: 0.8350 - val_loss: 0.3857 - val_acc: 0.8360
Epoch 4/25
 - 161s - loss: 0.3221 - acc: 0.8616 - val_loss: 0.3730 - val_acc: 0.8479
Epoch 5/25
 - 161s - loss: 0.2768 - acc: 0.8812 - val_loss: 0.3790 - val_acc: 0.8586
Train on 17120 samples, validate on 42

In [13]:
print (np.mean(val_loss_result))
res = (pred_results[0] + pred_results[1] + pred_results[2] +
     pred_results[3] + pred_results[4]) / 5
print (np.mean(res))

0.3773869845243258
0.19177021


In [14]:
train_pred_results

[array([0.00037221], dtype=float32),
 array([0.72075266], dtype=float32),
 array([0.00436131], dtype=float32),
 array([0.93674177], dtype=float32),
 array([0.01624144], dtype=float32),
 array([0.00352858], dtype=float32),
 array([0.06040532], dtype=float32),
 array([0.1742406], dtype=float32),
 array([0.00300759], dtype=float32),
 array([0.0013195], dtype=float32),
 array([0.00965152], dtype=float32),
 array([0.0456045], dtype=float32),
 array([0.04185075], dtype=float32),
 array([0.38411963], dtype=float32),
 array([0.5946873], dtype=float32),
 array([0.0003088], dtype=float32),
 array([0.07085063], dtype=float32),
 array([0.03532735], dtype=float32),
 array([0.00280013], dtype=float32),
 array([0.0064416], dtype=float32),
 array([0.04490587], dtype=float32),
 array([0.6251682], dtype=float32),
 array([0.01430059], dtype=float32),
 array([0.00134125], dtype=float32),
 array([0.00168582], dtype=float32),
 array([0.92724985], dtype=float32),
 array([0.9977374], dtype=float32),
 array([0

In [23]:
for i in range(res.shape[0]):
    res[i] = adj(res[i],0.3,0.25)

In [24]:
res

array([[0.5136753 ],
       [0.09629986],
       [0.3013578 ],
       ...,
       [0.9956596 ],
       [0.8070951 ],
       [0.9934422 ]], dtype=float32)

In [15]:
train_pred_results.extend(res)

In [16]:
train_pred_results = pd.DataFrame(train_pred_results)
#test['pred'] = res
#test[['pred']].to_csv('cnnresult.txt', index = None, header = None)
train_pred_results.to_csv('cnn2result.dat',index = None)

In [39]:
len(res)

5000

In [44]:
len(res[np.where(res == 0)])

0

In [54]:
len(np.unique(test['spa_qura1']))

3199

In [55]:
len(np.unique(test['spa_qura2']))

2695

In [56]:
len(np.unique(train['spa_qura1']))

3540

In [57]:
len(np.unique(train['spa_qura2']))

5979

In [8]:
def aveResult(result_array_list):
    final = sum(result_array_list)/len(result_array_list)
    print(np.mean(final))
    final = pd.DataFrame(final)
    final.to_csv('result_.txt', index = None, header = None)

In [ ]:
cv = [cv0,cv5]
aveResult(cv)

In [7]:
xgb = pd.read_csv('result&gbdt.txt',header = None)
xgb.describe()

,0
count,5000.000000
mean,0.248454
std,0.269926
min,0.004539
25%,0.056728
50%,0.129305
75%,0.337693
max,0.996406


In [2]:
np.mean(cv5)

0.24845433020000002

In [15]:
train = pd.read_csv('Are-you-happy/traditional_model/train.dat')
train.sentence_distance.head()

0    0.227921
1    0.336336
2    0.286039
3    0.204124
4    0.474342
Name: sentence_distance, dtype: float64

In [16]:
df = pd.DataFrame()
df['sentence_distance'] = train['sentence_distance']
df['sentence_distance_tfidf'] = train['sentence_distance_tfidf']
df.to_csv('sentence_sim_feature.dat',index = False)

In [1]:
import pandas as pd
import numpy as np
#df = pd.read_csv('train.dat')
#gbdt = pd.read_csv('result/result_gbdt.txt',header = None)
cnn_gbdt = pd.read_csv('../diff_traditional/result_gbdt.txt',header=None)
#gbdt_sim = pd.read_csv('result/result_gbdt&sim.txt',header=None)

In [20]:
aa = pd.read_csv('../diff_traditional/train.dat')

In [23]:
aa.sentence_similarity.head()

0    0.830081
1    0.894331
2    0.750192
3    0.608729
4    0.864632
Name: sentence_similarity, dtype: float64

In [4]:
gbdt_sim.mean()

0          0.244274
rescale    0.274023
dtype: float64

In [3]:
def adj(x, te, tr):
        a = te / tr
        b = (1 - te) / (1 - tr)
        return a * x / (a * x + b * (1 - x))

In [18]:
res = pd.DataFrame(res)

In [19]:
res['rescale'] = res.apply(lambda x:adj(x[0],0.5,0.25),axis=1)

In [20]:
res.mean()

0          0.207800
rescale    0.336195
dtype: float64

In [21]:
res['rescale'].to_csv('result_cnn2_rescale.txt',index = False, header = None)

In [4]:
#gbdt_sim['rescale'] = gbdt_sim.apply(lambda x:adj(x,0.3,0.25),axis=1)
cnn_gbdt['rescale'] = cnn_gbdt.apply(lambda x:adj(x[0],0.5,0.25),axis=1)

In [5]:
cnn_gbdt.mean()

0          0.239281
rescale    0.358570
dtype: float64

In [6]:
cnn_gbdt['rescale'].to_csv('result_gbdt_cnn2_rescale.txt',index = False,header = None)

In [42]:
gbdt_sim['rescale'].to_csv('result_gbdt_sim_rescale.txt',index = False,header = None)

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
df = pd.read_csv('../diff_traditional/train.dat')
distance_feature = df[['len_char_s1','len_char_s2','fuzz_QRatio','fuzz_WRatio','sentence_distance_tfidf',
    'cosine_distance_ave_idf','skew_s1vec_ave_idf', 'skew_s2vec_ave_idf', 'kur_s1vec_ave_idf',
       'kur_s2vec_ave_idf']]
distance_feature = np.array(distance_feature)

Index(['label', 'sentences', 'len_word_s1', 'len_word_s2', 'len_ratio',
       'len_char_s1', 'len_char_s2', 'fuzz_QRatio', 'fuzz_WRatio',
       'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'long_common_sequence', 'long_common_prefix',
       'long_common_suffix', 'long_common_substring', 'levenshtein_distance',
       'sentence_similarity', 'has_no_word', 'sentence_distance',
       'sentence_distance_tfidf', 'lcs_diff', 'cnn_result',
       '1-gramoverlap_word', '2-gramoverlap_word', '3-gramoverlap_word',
       '2-gramoverlap_char', '3-gramoverlap_char', '4-gramoverlap_char',
       '5-gramoverlap_char', 'jaccard_distance_ave_idf',
       'cosine_distance_ave_idf', 'canberra_distance_ave_idf',
       'euclidean_distance_ave_idf', 'braycurtis_distance_ave_idf',
       'minkowski_distance_ave_idf', 'pearson_coff_ave_idf',
       'spearman_coff_ave_idf', 'kendalltau_coff_ave_idf',


In [19]:
distance_feature = df[['len_char_s1','len_char_s2','fuzz_QRatio','fuzz_WRatio','sentence_distance_tfidf',
    'cosine_distance_ave_idf','skew_s1vec_ave_idf', 'skew_s2vec_ave_idf', 'kur_s1vec_ave_idf',
       'kur_s2vec_ave_idf']]

In [28]:
distance_feature = np.array(distance_feature)
distance_feature[:2]

array([[32.        , 62.        , 39.        , 86.        ,  0.22545252,
         0.25577446, -0.19645793, -0.35475846,  0.10816221,  0.52349916],
       [86.        , 67.        , 31.        , 54.        ,  0.19932241,
         0.12921127, -0.43639298, -0.35495612,  1.16359401,  0.8015316 ]])